In [1]:
from bs4 import BeautifulSoup
import requests
import re ## library to do regular expression matching
import pandas as pd

In [2]:
# making a page request: if its legal to scrape it, response 200 is ok

url = "https://www.atptour.com/en/rankings/singles?rankDate=2019-05-13&rankRange=1-5000"
page = requests.get(url,timeout=50) 

# if page.status_code == 200: continue..

### Convert data of single page into data frame

In [3]:
# convert page content into an object
content_obj = BeautifulSoup(page.content, "html.parser")

print(content_obj.prettify())

<!DOCTYPE html>
<!-- START : /modules/global/head -->
<!--[if lt IE 7]>
    <html class="no-js lt-ie10 lt-ie9 lt-ie8 lt-ie7 ">
<![endif]-->
<!--[if IE 7]>
    <html class="no-js lt-ie10 lt-ie9 lt-ie8 ">
<![endif]-->
<!--[if IE 8]>
    <html class="no-js lt-ie10 lt-ie9 ">
<![endif]-->
<!--[if IE 9]>
    <html class="no-js lt-ie10 ">
<![endif]-->
<!--[if gt IE 9]><!-->
<html class="no-js">
 <!--<![endif]-->
 <head>
  <!-- disable auto format for telephone numbers -->
  <meta content="telephone=no" name="format-detection"/>
  <title>
   Rankings | Singles | ATP Tour | Tennis
  </title>
  <meta content="initial-scale=1.0, width=768, user-scalable=yes, minimum-scale=1.0, maximum-scale=1.25" name="viewport"/>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="" name="keywords"/>
  <meta content="Tennis' official singles rankings of the ATP Tour, featuring Novak Djokovic, Roger Federer, Rafael Nadal, Alexander Zverev and more." name="d

In [4]:
# now move from top to bottom, check every step in between!
content_obj_1 = content_obj.find('body',{'class':'is-desktop'}).find('div',{'class':'wrapper'})
content_obj_2 = content_obj_1.find('div',{'class':'container'}).find('div',{'data-page':'singles'})
content_obj_3 = content_obj_2.find('div',{'class':'table-rankings'}).find('table',{'class':'mega-table'}).find('tbody')
# print(content_obj_3.prettify())

content_tr = content_obj_3.findAll('tr')
# contains all data for all rows:
print(content_tr[0].prettify)

#len(content_tr)
#type(content_tr)

<bound method Tag.prettify of <tr>
<td class="rank-cell">
					1
				</td>
<td class="move-cell">
<div class="move-none"></div>
<div class="move-text">
</div>
</td>
<td class="country-cell">
<div class="country-inner">
<div class="country-item">
<img alt="SRB" onerror="this.remove()" src="/en/~/media/images/flags/srb.svg"/>
</div>
</div>
</td>
<td class="player-cell">
<a data-ga-label="Novak Djokovic" href="/en/players/novak-djokovic/d643/overview">Novak Djokovic</a> </td>
<td class="age-cell">
31				</td>
<td class="points-cell">
<a data-ga-label="rankings-breakdown" href="/en/players/novak-djokovic/d643/rankings-breakdown?team=singles">12,115</a> </td>
<td class="tourn-cell">
<a data-ga-label="17" href="/en/players/novak-djokovic/d643/player-activity?matchType=singles">17</a> </td>
<td class="pts-cell">
						360
					</td>
<td class="next-cell">
						0
					</td>
</tr>>


In [5]:
# lop over all tr = entries, extract content, and clean it!
# cleaning: .strip(), or re.sub("\n","", data)

rank_list = []
moves_list = []
countries_list = []
player_names_list = []
age_list = []
points_list = []
tourn_list = []
pointsdrop_list = []
nextbest_list = []
date_list = []

for i in range(0,len(content_tr)):
    
    ranks = content_tr[i].find('td',{'class':'rank-cell'}).get_text().strip()
    rank_list.append(ranks) 
    
    moves = content_tr[i].find('td',{'class':'rank-cell'}).get_text().strip()
    moves_list.append(moves)
    
    countries_raw = content_tr[i].find('div',{'class':'country-item'}).find('img',{'alt':True})
    pattern = r'alt=\"(.*?)\"'
    countries = re.search(pattern, str(countries_raw)).group(1) # group(1) gives exact match
    countries_list.append(countries)
    
    player_names = content_tr[i].find('td',{'class':'player-cell'}).get_text().strip()
    player_names_list.append(player_names)
    
    ages = content_tr[i].find('td',{'class':'age-cell'}).get_text().strip()
    age_list.append(ages)
    
    points = content_tr[i].find('td',{'class':'points-cell'}).get_text().strip()
    points_list.append(points)
    
    tourns = content_tr[i].find('td',{'class':'tourn-cell'}).get_text().strip()
    tourn_list.append(tourns)
          
    pointsdrop = content_tr[i].find('td',{'class':'pts-cell'}).get_text().strip()
    pointsdrop_list.append(pointsdrop)
    
    nextbest = content_tr[i].find('td',{'class':'next-cell'}).get_text().strip()
    nextbest_list.append(nextbest)    
    

In [6]:
# convert lists into data frame
page_table = pd.DataFrame({
                    'Rank' : rank_list,
                    'Move' : moves_list,
                    'Country': countries_list,
                    'Player' : player_names_list,
                    'Age' : age_list,
                    'Points' : points_list,
                    'Tournaments' : tourn_list,
                    'Points_dropping' : pointsdrop_list,
                    'Next_best' : nextbest_list
                })

page_table

,Rank,Move,Country,Player,Age,Points,Tournaments,Points_dropping,Next_best
0,1,1,SRB,Novak Djokovic,31,"12,115",17,360,0
1,2,2,ESP,Rafael Nadal,32,"7,945",16,"1,000",0
2,3,3,SUI,Roger Federer,37,"5,770",17,0,0
3,4,4,AUT,Dominic Thiem,25,"4,845",24,10,90
4,5,5,GER,Alexander Zverev,22,"4,745",22,600,20
5,6,6,JPN,Kei Nishikori,29,"3,860",23,180,90
6,7,7,GRE,Stefanos Tsitsipas,20,"3,790",28,70,90
7,8,8,RSA,Kevin Anderson,32,"3,755",17,10,0
8,9,9,ARG,Juan Martin del Potro,30,"3,145",14,90,0
9,10,10,CRO,Marin Cilic,30,"3,025",19,360,0


### Generate links to data of all possible dates

In [7]:
# https://www.atptour.com/en/rankings/singles?rankDate=2019-05-13&rankRange=1-5000

# get dates:
content_obj_dates = content_obj_2.find('div',{'class':'main-content'}).find('div',{'class':'dropdown-holder'})
content_obj_dates_list = content_obj_dates.find('ul',{'data-value':'rankDate'}).findAll('li',{'data-value':True})

dates_list = []

for j in range(0,len(content_obj_dates_list)):
    curr_date = content_obj_dates_list[j].get_text().strip()
    curr_date_reformat = curr_date.replace(".","-")  # this is required for webpage format!
    dates_list.append(curr_date_reformat)
    
#print(dates_list)
len(dates_list)

2070

In [8]:
# now create all links for pages PER DATE:
# https://www.atptour.com/en/rankings/singles?rankDate=2019-05-13&rankRange=1-5000
urlpattern = "http://www.atpworldtour.com/en/rankings/singles?rankDate=%s&rankRange=1-5000"
links_list = []

for x in range(0,len(dates_list)):
    links_curr = urlpattern % (dates_list[x])
    links_list.append(links_curr)
    
print(links_list[0])

http://www.atpworldtour.com/en/rankings/singles?rankDate=2019-05-20&rankRange=1-5000


### Retrieve data from all links, append data frames

In [9]:
# pack all above code into function:

def scrape_ATP_page(link):
    '''This function retrieves the ATP table for given link'''
    
    page = requests.get(link,timeout=50) 
    content_obj = BeautifulSoup(page.content, "html.parser")
    
    # get date out of link
    datepattern = r'rankDate=(.*?)&rankRange'
    curr_date = re.search(datepattern, link).group(1) # group(1) gives exact match
    
    content_obj_1 = content_obj.find('body',{'class':'is-desktop'}).find('div',{'class':'wrapper'})
    content_obj_2 = content_obj_1.find('div',{'class':'container'}).find('div',{'data-page':'singles'})
    content_obj_3 = content_obj_2.find('div',{'class':'table-rankings'}).find('table',{'class':'mega-table'}).find('tbody')
    content_tr = content_obj_3.findAll('tr')
 
    rank_list = []
    moves_list = []
    #countries_list = []
    player_names_list = []
    age_list = []
    #points_list = []
    #tourn_list = []
    pointsdrop_list = []
    #nextbest_list = []
    
    for i in range(0,len(content_tr)):
    
        ranks = content_tr[i].find('td',{'class':'rank-cell'}).get_text().strip()
        rank_list.append(ranks) 
    
        moves = content_tr[i].find('td',{'class':'rank-cell'}).get_text().strip()
        moves_list.append(moves)
    
        #countries_raw = content_tr[i].find('div',{'class':'country-item'}).find('img',{'alt':True})
        #pattern = r'alt=\"(.*?)\"'
        #countries = re.search(pattern, str(countries_raw)).group(1) # group(1) gives exact match
        #countries_list.append(countries)
    
        player_names = content_tr[i].find('td',{'class':'player-cell'}).get_text().strip()
        player_names_list.append(player_names)
    
        ages = content_tr[i].find('td',{'class':'age-cell'}).get_text().strip()
        age_list.append(ages)
    
        #points = content_tr[i].find('td',{'class':'points-cell'}).get_text().strip()
        #points_list.append(points)
    
        #tourns = content_tr[i].find('td',{'class':'tourn-cell'}).get_text().strip()
        #tourn_list.append(tourns)
          
        pointsdrop = content_tr[i].find('td',{'class':'pts-cell'}).get_text().strip()
        pointsdrop_list.append(pointsdrop)
    
        #nextbest = content_tr[i].find('td',{'class':'next-cell'}).get_text().strip()
        #nextbest_list.append(nextbest) 
        
    page_table = pd.DataFrame({
                    'Date' : curr_date,
                    'Rank' : rank_list,
                    'Move' : moves_list,
                    #'Country': countries_list,
                    'Player' : player_names_list,
                    'Age' : age_list,
                    #'Points' : points_list,
                    #'Tournaments' : tourn_list,
                    'Points_dropping' : pointsdrop_list,
                    #'Next_best' : nextbest_list
                })

    return page_table

In [10]:
# test function 
link = links_list[1000]
test_table = scrape_ATP_page(link)
test_table

,Date,Rank,Move,Player,Age,Points_dropping
0,1999-04-05,1,1,Pete Sampras,27,0
1,1999-04-05,2,2,Carlos Moya,22,0
2,1999-04-05,3,3,Yevgeny Kafelnikov,25,0
3,1999-04-05,4,4,Richard Krajicek,27,0
4,1999-04-05,5,5,Patrick Rafter,26,0
5,1999-04-05,6,6,Alex Corretja,24,0
6,1999-04-05,7,7,Tim Henman,24,0
7,1999-04-05,8,8,Todd Martin,28,0
8,1999-04-05,9,9,Mark Philippoussis,22,0
9,1999-04-05,10,10,Karol Kucera,25,0


## ACHTUNG! the code runs for about 2.5 hours!!

In [11]:
columns = test_table.columns.values

BigFatDataSet = pd.DataFrame(columns=columns)

for y in range(0,len(links_list)):
    print('Retrieving data from page %d out of %d' % (y+1 , len(links_list)+1))
    page_table = scrape_ATP_page(links_list[y])
    BigFatDataSet = BigFatDataSet.append(page_table)


Retrieving data from page 1 out of 2071
Retrieving data from page 2 out of 2071
Retrieving data from page 3 out of 2071
Retrieving data from page 4 out of 2071
Retrieving data from page 5 out of 2071
Retrieving data from page 6 out of 2071
Retrieving data from page 7 out of 2071
Retrieving data from page 8 out of 2071
Retrieving data from page 9 out of 2071
Retrieving data from page 10 out of 2071
Retrieving data from page 11 out of 2071
Retrieving data from page 12 out of 2071
Retrieving data from page 13 out of 2071
Retrieving data from page 14 out of 2071
Retrieving data from page 15 out of 2071
Retrieving data from page 16 out of 2071
Retrieving data from page 17 out of 2071
Retrieving data from page 18 out of 2071
Retrieving data from page 19 out of 2071
Retrieving data from page 20 out of 2071
Retrieving data from page 21 out of 2071
Retrieving data from page 22 out of 2071
Retrieving data from page 23 out of 2071
Retrieving data from page 24 out of 2071
Retrieving data from page

Retrieving data from page 199 out of 2071
Retrieving data from page 200 out of 2071
Retrieving data from page 201 out of 2071
Retrieving data from page 202 out of 2071
Retrieving data from page 203 out of 2071
Retrieving data from page 204 out of 2071
Retrieving data from page 205 out of 2071
Retrieving data from page 206 out of 2071
Retrieving data from page 207 out of 2071
Retrieving data from page 208 out of 2071
Retrieving data from page 209 out of 2071
Retrieving data from page 210 out of 2071
Retrieving data from page 211 out of 2071
Retrieving data from page 212 out of 2071
Retrieving data from page 213 out of 2071
Retrieving data from page 214 out of 2071
Retrieving data from page 215 out of 2071
Retrieving data from page 216 out of 2071
Retrieving data from page 217 out of 2071
Retrieving data from page 218 out of 2071
Retrieving data from page 219 out of 2071
Retrieving data from page 220 out of 2071
Retrieving data from page 221 out of 2071
Retrieving data from page 222 out 

Retrieving data from page 395 out of 2071
Retrieving data from page 396 out of 2071
Retrieving data from page 397 out of 2071
Retrieving data from page 398 out of 2071
Retrieving data from page 399 out of 2071
Retrieving data from page 400 out of 2071
Retrieving data from page 401 out of 2071
Retrieving data from page 402 out of 2071
Retrieving data from page 403 out of 2071
Retrieving data from page 404 out of 2071
Retrieving data from page 405 out of 2071
Retrieving data from page 406 out of 2071
Retrieving data from page 407 out of 2071
Retrieving data from page 408 out of 2071
Retrieving data from page 409 out of 2071
Retrieving data from page 410 out of 2071
Retrieving data from page 411 out of 2071
Retrieving data from page 412 out of 2071
Retrieving data from page 413 out of 2071
Retrieving data from page 414 out of 2071
Retrieving data from page 415 out of 2071
Retrieving data from page 416 out of 2071
Retrieving data from page 417 out of 2071
Retrieving data from page 418 out 

Retrieving data from page 591 out of 2071
Retrieving data from page 592 out of 2071
Retrieving data from page 593 out of 2071
Retrieving data from page 594 out of 2071
Retrieving data from page 595 out of 2071
Retrieving data from page 596 out of 2071
Retrieving data from page 597 out of 2071
Retrieving data from page 598 out of 2071
Retrieving data from page 599 out of 2071
Retrieving data from page 600 out of 2071
Retrieving data from page 601 out of 2071
Retrieving data from page 602 out of 2071
Retrieving data from page 603 out of 2071
Retrieving data from page 604 out of 2071
Retrieving data from page 605 out of 2071
Retrieving data from page 606 out of 2071
Retrieving data from page 607 out of 2071
Retrieving data from page 608 out of 2071
Retrieving data from page 609 out of 2071
Retrieving data from page 610 out of 2071
Retrieving data from page 611 out of 2071
Retrieving data from page 612 out of 2071
Retrieving data from page 613 out of 2071
Retrieving data from page 614 out 

Retrieving data from page 787 out of 2071
Retrieving data from page 788 out of 2071
Retrieving data from page 789 out of 2071
Retrieving data from page 790 out of 2071
Retrieving data from page 791 out of 2071
Retrieving data from page 792 out of 2071
Retrieving data from page 793 out of 2071
Retrieving data from page 794 out of 2071
Retrieving data from page 795 out of 2071
Retrieving data from page 796 out of 2071
Retrieving data from page 797 out of 2071
Retrieving data from page 798 out of 2071
Retrieving data from page 799 out of 2071
Retrieving data from page 800 out of 2071
Retrieving data from page 801 out of 2071
Retrieving data from page 802 out of 2071
Retrieving data from page 803 out of 2071
Retrieving data from page 804 out of 2071
Retrieving data from page 805 out of 2071
Retrieving data from page 806 out of 2071
Retrieving data from page 807 out of 2071
Retrieving data from page 808 out of 2071
Retrieving data from page 809 out of 2071
Retrieving data from page 810 out 

...

Retrieving data from page 1174 out of 2071
Retrieving data from page 1175 out of 2071
Retrieving data from page 1176 out of 2071
Retrieving data from page 1177 out of 2071
Retrieving data from page 1178 out of 2071
Retrieving data from page 1179 out of 2071
Retrieving data from page 1180 out of 2071
Retrieving data from page 1181 out of 2071
Retrieving data from page 1182 out of 2071
Retrieving data from page 1183 out of 2071
Retrieving data from page 1184 out of 2071
Retrieving data from page 1185 out of 2071
Retrieving data from page 1186 out of 2071
Retrieving data from page 1187 out of 2071
Retrieving data from page 1188 out of 2071
Retrieving data from page 1189 out of 2071
Retrieving data from page 1190 out of 2071
Retrieving data from page 1191 out of 2071
Retrieving data from page 1192 out of 2071
Retrieving data from page 1193 out of 2071
Retrieving data from page 1194 out of 2071
Retrieving data from page 1195 out of 2071
Retrieving data from page 1196 out of 2071
Retrieving 

Retrieving data from page 1365 out of 2071
Retrieving data from page 1366 out of 2071
Retrieving data from page 1367 out of 2071
Retrieving data from page 1368 out of 2071
Retrieving data from page 1369 out of 2071
Retrieving data from page 1370 out of 2071
Retrieving data from page 1371 out of 2071
Retrieving data from page 1372 out of 2071
Retrieving data from page 1373 out of 2071
Retrieving data from page 1374 out of 2071
Retrieving data from page 1375 out of 2071
Retrieving data from page 1376 out of 2071
Retrieving data from page 1377 out of 2071
Retrieving data from page 1378 out of 2071
Retrieving data from page 1379 out of 2071
Retrieving data from page 1380 out of 2071
Retrieving data from page 1381 out of 2071
Retrieving data from page 1382 out of 2071
Retrieving data from page 1383 out of 2071
Retrieving data from page 1384 out of 2071
Retrieving data from page 1385 out of 2071
Retrieving data from page 1386 out of 2071
Retrieving data from page 1387 out of 2071
Retrieving 

Retrieving data from page 1556 out of 2071
Retrieving data from page 1557 out of 2071
Retrieving data from page 1558 out of 2071
Retrieving data from page 1559 out of 2071
Retrieving data from page 1560 out of 2071
Retrieving data from page 1561 out of 2071
Retrieving data from page 1562 out of 2071
Retrieving data from page 1563 out of 2071
Retrieving data from page 1564 out of 2071
Retrieving data from page 1565 out of 2071
Retrieving data from page 1566 out of 2071
Retrieving data from page 1567 out of 2071
Retrieving data from page 1568 out of 2071
Retrieving data from page 1569 out of 2071
Retrieving data from page 1570 out of 2071
Retrieving data from page 1571 out of 2071
Retrieving data from page 1572 out of 2071
Retrieving data from page 1573 out of 2071
Retrieving data from page 1574 out of 2071
Retrieving data from page 1575 out of 2071
Retrieving data from page 1576 out of 2071
Retrieving data from page 1577 out of 2071
Retrieving data from page 1578 out of 2071
Retrieving 

Retrieving data from page 1747 out of 2071
Retrieving data from page 1748 out of 2071
Retrieving data from page 1749 out of 2071
Retrieving data from page 1750 out of 2071
Retrieving data from page 1751 out of 2071
Retrieving data from page 1752 out of 2071
Retrieving data from page 1753 out of 2071
Retrieving data from page 1754 out of 2071
Retrieving data from page 1755 out of 2071
Retrieving data from page 1756 out of 2071
Retrieving data from page 1757 out of 2071
Retrieving data from page 1758 out of 2071
Retrieving data from page 1759 out of 2071
Retrieving data from page 1760 out of 2071
Retrieving data from page 1761 out of 2071
Retrieving data from page 1762 out of 2071
Retrieving data from page 1763 out of 2071
Retrieving data from page 1764 out of 2071
Retrieving data from page 1765 out of 2071
Retrieving data from page 1766 out of 2071
Retrieving data from page 1767 out of 2071
Retrieving data from page 1768 out of 2071
Retrieving data from page 1769 out of 2071
Retrieving 

Retrieving data from page 1938 out of 2071
Retrieving data from page 1939 out of 2071
Retrieving data from page 1940 out of 2071
Retrieving data from page 1941 out of 2071
Retrieving data from page 1942 out of 2071
Retrieving data from page 1943 out of 2071
Retrieving data from page 1944 out of 2071
Retrieving data from page 1945 out of 2071
Retrieving data from page 1946 out of 2071
Retrieving data from page 1947 out of 2071
Retrieving data from page 1948 out of 2071
Retrieving data from page 1949 out of 2071
Retrieving data from page 1950 out of 2071
Retrieving data from page 1951 out of 2071
Retrieving data from page 1952 out of 2071
Retrieving data from page 1953 out of 2071
Retrieving data from page 1954 out of 2071
Retrieving data from page 1955 out of 2071
Retrieving data from page 1956 out of 2071
Retrieving data from page 1957 out of 2071
Retrieving data from page 1958 out of 2071
Retrieving data from page 1959 out of 2071
Retrieving data from page 1960 out of 2071
Retrieving 

### reset index and save as csv and json file

In [13]:
BigFatDataSet.reset_index(drop=True)
BigFatDataSet.to_csv('ATP_newdata.csv', encoding='utf-8', index=False)
with open('ATP_newdata.json', 'w') as f:
    f.write(BigFatDataSet.to_json(orient='records', lines=True))